In [1]:
import sklearn
import numpy as np
import pandas as pd
from dev import DEVALGO 
from smote import SMOTEBoost
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score

In [2]:
MINORITY = 1
MAJORITY = 0
CLASS_NAME = "Class"

In [3]:
def pprint_dict(d):
    for key, value in d.items():
        print(key + ": " + "{0:.2f}".format(value/100.0))

In [4]:
df = pd.read_csv("data/Blood/blood.csv").sample(frac=1, random_state=0).reset_index(drop=True)
df.head()

,V1,V2,V3,V4,Class
0,4,3,750,28,1
1,2,11,2750,41,2
2,2,6,1500,15,2
3,4,10,2500,28,2
4,2,41,10250,98,2


In [5]:
# Outcomes distribution
df[CLASS_NAME].value_counts()

1    570
2    178
Name: Class, dtype: int64

In [6]:
# Uncomment this when testing on vowel
df[CLASS_NAME].replace({1:0, 2:1}, inplace=True)
df.head()

,V1,V2,V3,V4,Class
0,4,3,750,28,0
1,2,11,2750,41,1
2,2,6,1500,15,1
3,4,10,2500,28,1
4,2,41,10250,98,1


In [7]:
# Calculate imbalanced ratio
frq = df[CLASS_NAME].value_counts()
ratio = max(frq)/min(frq)
"{0:.1f}".format(ratio)

'3.2'

# Use the following for testing Decision tree

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [8]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train, y_train)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [9]:
pprint_dict(res)

precision: 0.69
recall: 0.72
f1-score: 0.70
specificity: 0.86
sensitivity: 0.28
overall accuracy: 0.72
auc: 0.57
g_mean: 0.49


# Use the following for testing SMOTE

- run 100 iterations
- 50/50 training/testing
- Decision tree
- N = 200

In [10]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    if i == 0:
        print("Shape of X_train before oversampling: " + str(X_train.shape))
        print("Outcome distribution of X_train before oversampling: " + str(np.bincount(y_train)))
    # Oversample training data
    sm = SMOTE(random_state=0)
    sm.fit(X_train, y_train)
    X_train_r, y_train_r = sm.sample(X_train, y_train)
    if i == 0:
        print("Shape of X_train after oversampling: " + str(X_train_r.shape))
        print("Outcome distribution of X_train after oversampling: " + str(np.bincount(y_train_r)))
    # Build classifier on resampled data
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train_r, y_train_r)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

Shape of X_train before oversampling: (374, 4)
Outcome distribution of X_train before oversampling: [286  88]
Shape of X_train after oversampling: (572, 4)
Outcome distribution of X_train after oversampling: [286 286]


In [11]:
pprint_dict(res)

precision: 0.71
recall: 0.71
f1-score: 0.71
specificity: 0.81
sensitivity: 0.39
overall accuracy: 0.71
auc: 0.60
g_mean: 0.56


# Use the following for testing ADASYN

- run 100 iterations
- 50/50 training/testing
- Decision tree
- A fully balanced dataset after synthesizing
- Dth = 0.75 (Dth is a preset threshold for the maximum tolerated degree of class imbalance ratio)

In [12]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    if i == 0:
        print("Shape of X_train before oversampling: " + str(X_train.shape))
        print("Outcome distribution of X_train before oversampling: " + str(np.bincount(y_train)))
    # Oversample training data
    ada = ADASYN(random_state=0)
    ada.fit(X_train, y_train)
    X_train_r, y_train_r = ada.sample(X_train, y_train)
    if i == 0:
        print("Shape of X_train after oversampling: " + str(X_train_r.shape))
        print("Outcome distribution of X_train after oversampling: " + str(np.bincount(y_train_r)))
    # Build classifier on resampled data
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train_r, y_train_r)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

Shape of X_train before oversampling: (374, 4)
Outcome distribution of X_train before oversampling: [286  88]
Shape of X_train after oversampling: (567, 4)
Outcome distribution of X_train after oversampling: [286 281]


In [13]:
pprint_dict(res)

precision: 0.67
recall: 0.68
f1-score: 0.68
specificity: 0.80
sensitivity: 0.30
overall accuracy: 0.68
auc: 0.55
g_mean: 0.49


# Use the following for testing SMOTEBoost

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [14]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf1 = SMOTEBoost(random_state=0)
    clf1.fit(X_train, y_train)
    y_pred = clf1.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [15]:
pprint_dict(res)

precision: 0.78
recall: 0.44
f1-score: 0.44
specificity: 0.29
sensitivity: 0.93
overall accuracy: 0.44
auc: 0.61
g_mean: 0.52


# Use the following for testing Dev_algo

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [18]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf1 = DEVALGO(random_state=0, n_samples=392)
    clf1.fit(X_train, y_train)
    y_pred = clf1.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["weighted avg"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["weighted avg"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["weighted avg"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [19]:
 pprint_dict(res)

precision: 0.71
recall: 0.45
f1-score: 0.47
specificity: 0.34
sensitivity: 0.81
overall accuracy: 0.45
auc: 0.58
g_mean: 0.53
